In [1]:
import os
import re
import tqdm

from bs4 import BeautifulSoup as bs

import pandas as pd

## create file map

In [3]:
dp_in = '/media/rtn/Windows 10/work/univier/wiki_extract/wiki_db'
dp_out = '/media/rtn/Windows 10/work/univier/wiki_extract/wiki_parsed'

In [4]:
df = []

for (dp, dns, fns) in os.walk(dp_in):
    for fn in fns:
        fp_orig = os.path.join(dp, fn)
        fp_parsed = os.path.join(dp_out, os.path.relpath(fp_orig, dp_in) + '.txt')
        df.append({'filename': fn, 'path': fp_parsed, 'html_path': fp_orig})
    
df = pd.DataFrame(df)
out_file = os.path.join(dp_out, 'filepaths.csv')
df.to_csv(out_file, index=False)

In [7]:
df = pd.read_csv(os.path.join(dp_out, 'filepaths.csv'))
print(df.shape)

(223619, 3)


In [6]:
df.sample(10)

filename  \
191685                       synod_of_dort   
66484                        escape_artist   
31229                          boris_dubin   
8231                        acequia__idaho   
59354                       donald_j__cram   
172953                         sacred_site   
6287                                390_bc   
114880  legislative_assembly_of_costa_rica   
36902                        camilla_wicks   
35563                       buzancy__aisne   

                                                     path  \
191685  /media/rtn/Windows 10/work/univier/wiki_extrac...   
66484   /media/rtn/Windows 10/work/univier/wiki_extrac...   
31229   /media/rtn/Windows 10/work/univier/wiki_extrac...   
8231    /media/rtn/Windows 10/work/univier/wiki_extrac...   
59354   /media/rtn/Windows 10/work/univier/wiki_extrac...   
172953  /media/rtn/Windows 10/work/univier/wiki_extrac...   
6287    /media/rtn/Windows 10/work/univier/wiki_extrac...   
114880  /media/rtn/Windows 10/work/univier/wiki_extrac...   
36902   /media/rtn/Windows 10/work/univier/wiki_extrac...   
35563   /media/rtn/Windows 10/work/univier/wiki_extrac...   

                                                html_path  
191685  /media/rtn/Windows 10/work/univier/wiki_extrac...  
66484   /media/rtn/Windows 10/work/univier/wiki_extrac...  
31229   /media/rtn/Windows 10/work/univier/wiki_extrac...  
8231    /media/rtn/Windows 10/work/univier/wiki_extrac...  
59354   /media/rtn/Windows 10/work/univier/wiki_extrac...  
172953  /media/rtn/Windows 10/work/univier/wiki_extrac...  
6287    /media/rtn/Windows 10/work/univier/wiki_extrac...  
114880  /media/rtn/Windows 10/work/univier/wiki_extrac...  
36902   /media/rtn/Windows 10/work/univier/wiki_extrac...  
35563   /media/rtn/Windows 10/work/univier/wiki_extrac...

## parse documents

In [110]:
class HtmlParser:
    def remove_redundant_tags(self, soup: bs):
        selectors = ['.infobox', '.navbox', '.catlinks', '.reflist', '#toc', 'table']
        for s in selectors:
            for tag in soup.select(s):
                tag.extract();
        
        # as we include <ul> tag we'd like to ignore lists under following sections:
        sections_to_ignore_ul = ['#Related_pages', '#Other_websites']
        for s in sections_to_ignore_ul:
            # remove list that is a 1st next sibling 
            # after h2 tag that is parent for span with corresponding id.
            tag_other_websites = soup.select(s)
            if tag_other_websites:
                next_tag = tag_other_websites[0].parent.find_next_siblings()
                if next_tag:
                    next_tag = next_tag[0]
                    if next_tag.name == 'ul':
                        next_tag.extract()
        
    def filter_tags(self, tag) -> bool:
        if tag.name not in ['p', 'dt', 'ul']:
            return False
        return True
    
    def postprocess(self, text) -> str:
        text = re.sub('\s+', ' ', text)  # replace sequential whitespaces
        text = re.sub('\[\d+\]', '', text)  # remove [1], [15], etc.
        return text
    
    def parse(self, html_text):
        soup = bs(html_text)
        self.remove_redundant_tags(soup)
        filtered_tags = soup.find_all(self.filter_tags)
        text_chunks = [x.get_text().strip() for x in filtered_tags]
        text = ' '.join(text_chunks)
        text = self.postprocess(text)
        return text

In [111]:
parser = HtmlParser()

In [130]:
# fp = df.loc[df['filename'] == 'belarus', 'html_path'].iloc[0]
# fp = df.loc[df['filename'] == 'bel_canto', 'html_path'].iloc[0]
# fp = df.loc[df['filename'] == 'bellini', 'html_path'].iloc[0]
# fp = df.loc[df['filename'] == '390_bc', 'html_path'].iloc[0]
# fp = df.loc[df['filename'] == 'belly_dance', 'html_path'].iloc[0]
# fp = df.loc[df['filename'] == 'france', 'html_path'].iloc[0]
fp = df.loc[df['filename'] == 'music', 'html_path'].iloc[0]

In [131]:
with open(fp) as fin: file = fin.read()
text = parser.parse(file)

In [132]:
print(text)

Music is a form of art that uses sound organised in time. Music is also a form of entertainment that puts sounds together in a way that people like, find interesting or dance to. Most music includes people singing with their voices or playing musical instruments, such as the piano, guitar, drums or violin. The word music comes from the Greek word (mousike), which means "(art) of the Muses". In Ancient Greece the Muses included the goddesses of music, poetry, art, and dance. Someone who makes music is known as a musician. Music is sound that has been organized by using rhythm, melody or harmony. If someone bangs saucepans while cooking, it makes noise. If a person bangs saucepans or pots in a rhythmic way, they are making a simple type of music. There are four things which music has most of the time: Music often has pitch. This means high and low notes. Tunes are made of notes that go up or down or stay on the same pitch. Music often has rhythm. Rhythm is the way the musical sounds and 